# Защита персональных данных клиентов

<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li><li><span><a href="#Общий-вывод" data-toc-modified-id="Общий-вывод-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Общий вывод</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

**Заказчик:** страховая компания «Хоть потоп».

**Цель исследования:** защитить данные клиентов.

**Задачи:**  
1. Разработать метод шифрования данных, при котором качество моделей машинного обучения не ухудшится.
2. Обосновать корректность работы данного метода.

**Исходные данные:**  
Исходные данные представлены в датасете insurance.csv - данные о клиентах компании.

В датасете содержатся следующие признаки:  
- Пол;
- Возраст;
- Зарплата;
- Члены семьи;
- Страховые выплаты.

**Условные обозначения:**

$P$ - произвольная обратимая матрица;  
$X$ - матрица признаков;  
$X_p$ - модифицированная матрица признаков;  
$w$ - вектор весов признаков;  
$w_p$ - вектор весов признаков для модифицированной матрицы признаков;  
$w0$ - вектор линейных смещений признаков;  
$a$ - вектор предсказаний для исходной матрицы;  
$a_p$ - вектор предсказаний для модифицированной матрицы;  
$y$ - вектор целевого признака;  

<div class="alert alert-success">
<font size="5"><b>Комментарий ревьюера</b></font>

Успех 👍:


Вступление в работу очень важно, так человек, который смотрит твой проект (и на работе в том числе) будет сразу введен в курс дела.





</div>

## Загрузка данных

Импортируем библиотеки:

In [1]:
import numpy as np
import pandas as pd
import ydata_profiling
from scipy.spatial import distance
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

D:\Programs\anaconda3\Lib\site-packages\numba\core\decorators.py:262: NumbaDeprecationWarning: numba.generated_jit is deprecated. Please see the documentation at: https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-generated-jit for more information and advice on a suitable replacement.
  warnings.warn(msg, NumbaDeprecationWarning)
C:\Users\bond_\AppData\Roaming\Python\Python311\site-packages\visions\backends\shared\nan_handling.py:50: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @nb.jit


Импортируем данные:

In [2]:
try:
    df = pd.read_csv('/datasets/insurance.csv')
except:
    df = pd.read_csv('https://code.s3.yandex.net/datasets/insurance.csv')

Выводим информацию по датафрейму:

In [3]:
df.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [4]:
# df.profile_report()

In [5]:
def inf(df_info):
    print()
    print('Общая информация:')
    display(df_info.info(memory_usage='deep'))
    print('_' * 50)
    print()
    print('Описательная статистика числовых значений:')
    display(df_info.describe().T)
    print('_' * 50)
    print()
    print('Процент пропущенных значений:')
    display(df_info.isna().mean()*100)
    print('_' * 50)
    print()
    print('Случайная выборка датафрейма:')
    display(df_info.sample(n = 5, random_state = 0))
    print('_' * 100)

In [6]:
inf(df)


Общая информация:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


None

__________________________________________________

Описательная статистика числовых значений:


,count,mean,std,min,25%,50%,75%,max
Пол,5000.0,0.4990,0.500049,0.0,0.0,0.0,1.0,1.0
Возраст,5000.0,30.9528,8.440807,18.0,24.0,30.0,37.0,65.0
Зарплата,5000.0,39916.3600,9900.083569,5300.0,33300.0,40200.0,46600.0,79000.0
Члены семьи,5000.0,1.1942,1.091387,0.0,0.0,1.0,2.0,6.0
Страховые выплаты,5000.0,0.1480,0.463183,0.0,0.0,0.0,0.0,5.0


__________________________________________________

Процент пропущенных значений:


Пол                  0.0
Возраст              0.0
Зарплата             0.0
Члены семьи          0.0
Страховые выплаты    0.0
dtype: float64

__________________________________________________

Случайная выборка датафрейма:


,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
398,0,30.0,37700.0,1,0
3833,1,29.0,43700.0,1,0
4836,1,38.0,43600.0,1,0
4572,1,27.0,43400.0,1,0
636,0,26.0,36100.0,1,0


____________________________________________________________________________________________________


**Вывод:**
1. В датафрейме 5000 строк и 5 столбцов. Один столбец содержит категориальные данные, четыре - числовые.
2. Пропущенных значений нет.
3. В датафрейме 147 (2.9%) дубликатов. Поскольку в датафрейме среди признаков отсутствует идентификатор клиента, нет оснований предполагать, что дублирующиеся строки относятся к одному и тому же клиенту. Дубликаты целесообразно оставить без изменений.
4. Возраст имеет высокую корреляцию с количеством страховых выплат - 0.549.
5. У 30.3% клиентов не указаны члены семьи.
6.  Большинство клиентов - 4436 (88.7%) - не получали страховых выплат. 

## Умножение матриц

Модифицированная матрица признаков - умножаем исходную матрицу признаков на обратимую матрицу:  

$$
X_p = X P
$$

Задача обучения - минимизировать функцию потерь $MSE$:

$$
w = \arg\min_w MSE(X w, y)
$$

$$
w_p = \arg\min_w MSE(X_p w_p, y)
$$

Формулы предсказания для исходной и модифицированной матриц признаков:

$$
a = Xw + w0
$$

$$
a_p = X_pw_p + w0_p
$$

Упрощенные формулы предсказания для исходной и модифицированной матриц признаков ($w0$ учтено в $w$):

$$
a = Xw
$$

$$
a_p = X_pw_p
$$


Формулы обучения для исходной и модифицированной матриц признаков:

$$
w = (X^T X)^{-1} X^T y
$$

$$
w_p = (X_p^T X_p)^{-1} X_p^T y
$$


В формулу $w_p$ вместо $X_p$ подставляем $X P$:  

$$
w_p = ((X P)^T X P)^{-1} (X P)^T y
$$

Раскрываем скобки $(X P)^T$:

$$
w_p = (P^T (X^T X) P)^{-1} P^T X^T y
$$

Раскрываем скобки $(P^T (X^T X) P)^{-1}$:

$$
w_p = P^{-1} (X^T X)^{-1} (P^T)^{-1} P^T X^T y = P^{-1} (X^T X)^{-1} E X^T y = P^{-1} (X^T X)^{-1} X^T y
$$

Из формулы выше видно $(X^T X)^{-1} X^T y = w$:

$$
w_p = P^{-1} w
$$

Подставляем $X_p$ и $w_p$ в формулу $a_p$:

$$
a_p = X_pw_p = X P P^{-1} w = X E w = X w = a
$$

**Вывод:**  
Умножение матрицы признаков на произвольную обратимую матрицу не ведет к изменению качества модели линейной регрессии.

## Алгоритм преобразования 

1. Выделяемиз датафрейма целевой признак в отдельный вектор.

2. Разделяем выборку на обучающую и тестовую в соотношении 75:25 методом train_test_split из библиотеки sklearn.

3. Определяем размер произвольной квадратной матрицы.  

Две матрицы можно перемножить только если количество столбцов у первой матрицы равно количеству строк второй.

$$
X_{(m×n)}⋅P_{(n×k)}=X_{p(m×k)}
$$

В исходном датафрейме 5 признаков, один которых - целевой. Следовательно, модель будет обучаться на векторе целевого признака и матрице, состоящей из 4 признаков. То есть матрица признаков будет иметь размер 5000x4. Из этого следует, произвольная обратимая матрица должна иметь размер 4x4:

$$
X_{(5000×4)}⋅P_{(4×4)}=X_{p(5000×4)}
$$

4. Создаем обратимую матрицу P методом random.rand из библиотеки numpy. Размеры данной матрицы - количество столбцов в матрице признаков.


Модифицированная матрица будет выражаться через исходную согласно равенству:


$$
X_p = X P
$$ 

5. Для исходной и модифицированной матриц признаков значения предсказаний рассчитываются по формулам:  

$$
a = X w + w0 $$
$$
a_p = X_p w_p + w0_p
$$

Для упрощения расчётов добавляем в матрицу $w$ нулевой столбец, заполненный единицами и соответствующий $w0$. Таким образом, формулы расчёта предсказаний приобретают следующий вид:  
    
$$
a = X w
$$

$$
a_p = X_p w_p
$$ 

6. Определяем веса признаков, при которых функция потери $MSE$ имеет минимальное значение: 

$$
w = (X^T X)^{-1} X^T y
$$ 

$$
w_m = (X_p^T X_p)^{-1} X_p^T y
$$


7. Из полученных матриц признаков выделяем первые столбцы, соответствующие $w0$ и $w0m$, и рассчитываем предсказания по исходным формулам:  

$$
a = X w + w0
$$

$$
a_p = X_p w_p + w0_p
$$ 

8. Рассчитываем метрики R2 и MSE на тестовых выборках.

## Проверка алгоритма

Переименовываем и переводим в нижний регистр названия признаков:

In [7]:
df = df.rename(columns = {'Пол':'sex',
                          'Возраст':'age',
                          'Зарплата':'salary',
                          'Члены семьи':'family_member',
                          'Страховые выплаты':'insurance'})

Разделяет датафрейм на признаки и целевой признак:

In [8]:
features = df.drop('insurance', axis=1)
targets = df['insurance']

Разделяем выборку на обучающую и тестовую:

In [9]:
features_train, features_test, target_train, target_test = train_test_split(features, targets, test_size=0.25, random_state=42)
features_train = features_train.reset_index(drop=True)
features_test = features_test.reset_index(drop=True)
target_train = target_train.reset_index(drop=True)
target_test = target_test.reset_index(drop=True)

Создаем матрицу и проверяем ее на обратимость:

In [10]:
def get_rand_matrix(dim):
    transform_matrix = np.random.rand(dim, dim)
    try: 
        inv = np.linalg.inv(transform_matrix)
        return transform_matrix
    except:
        func(dim) 

In [11]:
matrix = get_rand_matrix(features.shape[1])

Умножаем признаки обучающей и тестовой выборок на матрицу "matrix":

In [12]:
features_train_mod = pd.DataFrame(np.dot(features_train, matrix), columns=features_train.columns)
features_test_mod = pd.DataFrame(np.dot(features_test, matrix), columns=features_test.columns)

Класс линейной регрессии:

In [13]:
class IsuranceRegression:
    def fit(self, train_features, train_target):
        '''Функция обучения'''
        # создаем матрицу признаков (нулевой столбец состоит из единиц)
        X = np.concatenate((np.ones((train_features.shape[0], 1)), train_features), axis=1) 
        y = train_target                                            # вектор целевого признака
        w = (np.linalg.inv(X.T.dot(X))).dot(X.T).dot(y)             # вектор всех весов линейной регрессии
        self.w = w[1:]                                              # вектор весов признаков
        self.w0 = w[0]                                              # вектор веса линейного сдвига

    def predict(self, test_features):
        '''Функция предсказания и оценки'''
        predictions = test_features.dot(self.w) + self.w0           # предсказания для каждого объекта на основе w и w0
        r2 = r2_score(target_test, predictions).round(4)            # метрика r2
        mse = mean_squared_error(target_test, predictions).round(4) # метрика mse
        print('R2: {}'.format(r2))
        print('MSE: {}'.format(mse))
        return r2, mse 

Обучаем модель на исходной матрице признаков и оцениваем ее:

In [14]:
model = IsuranceRegression()
model.fit(features_train, target_train)
r2, mse = model.predict(features_test)

R2: 0.4255
MSE: 0.1386


Обучаем модель на модифицированной матрице признаков и оцениваем ее:

In [15]:
model_mod = IsuranceRegression()
model.fit(features_train_mod, target_train)
r2_matrix, mse_matrix = model.predict(features_test_mod)

R2: 0.4255
MSE: 0.1386


Сравниваем метрики моделей:

In [16]:
score_df = pd.DataFrame({'Признаки' : ['Исходные признаки', 'Модифицированные признаки'],
                         'R2' : [r2, r2_matrix],
                         'MSE' : [mse, mse_matrix]})
display(score_df)

if (r2 == r2_matrix) & (mse == mse_matrix):
    print('Умножение матрицы признаков на произвольную обратимую матрицу не влияет на качество модели линейной регрессии.')
else:
    print('Умножение матрицы признаков на произвольную обратимую матрицу влияет на качестве модели линейной регрессии.')

,Признаки,R2,MSE
0,Исходные признаки,0.4255,0.1386
1,Модифицированные признаки,0.4255,0.1386


Умножение матрицы признаков на произвольную обратимую матрицу не влияет на качество модели линейной регрессии.


## Общий вывод

**Информация по датафрейму**
1. В датафрейме 5000 строк и 5 столбцов. Один столбец содержит категориальные данные, четыре - числовые.
2. Пропущенных значений нет.
3. В датафрейме 147 (2.9%) дубликатов. Поскольку в датафрейме среди признаков отсутствует идентификатор клиент, нет оснований предполагать, что дублирующиеся строки относятся к одному и тому же клиенту. Дубликаты целесообразно оставить без изменений.
4. Возраст имеет высокую корреляцию с количеством страховых выплат - 0.549.
5. У 30.3% клиентов не указаны члены семьи.
6.  Большинство клиентов - 4436 (88.7%) - не получали страховых выплат. 

**Сравнение моделей**  
Теоретический анализ показал, что качество модели линейной регрессии при умножении матрицы признаков на произвольную обратимую матрицу остается неизменным.  
Проверка на исходном датафрейме подтвердила этот вывод - метрики моделей линейной регрессии, полученные на исходной и модифицируемой матрицах признаков, идентичны:  
- R2: 0.4255;
- MSE: 0.1386.